# 사전 작업

In [1]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import sys

# 코랩인지 확인하는 꿀팁
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    # 코랩용 경로
    BASE_PATH = '/content/drive/MyDrive/U-sto_AI'
else:
    # 로컬(내 컴퓨터)용 경로 (현재 폴더)
    BASE_PATH = os.getcwd()

print(f"📂 작업 경로 설정됨: {BASE_PATH}")

# 이동할 때도 이렇게!
try:
    os.chdir(BASE_PATH)
except FileNotFoundError:
    print("경로를 찾을 수 없습니다.")

# 라이브러리 설치
if os.path.exists("requirements.txt"):
    print("📄 requirements.txt를 감지했습니다. 버전에 맞춰 설치합니다.")
    !pip install -r requirements.txt
else:
    print("⚠️ requirements.txt가 없습니다. 최신 버전으로 설치합니다.")
    !pip install openai langchain langchain-community chromadb tiktoken

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📂 작업 경로 설정됨: /content/drive/MyDrive/U-sto_AI
📄 requirements.txt를 감지했습니다. 버전에 맞춰 설치합니다.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# OPENAI 연결 확인용 테스트
from google.colab import userdata
from openai import OpenAI

# 저장해둔 API 키 불러오기
api_key = userdata.get('OPENAI_API_KEY')

# 클라이언트 생성
client = OpenAI(api_key=api_key)

# 테스트 요청 (연결 확인용)
try:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": "안녕, 연결 테스트 중이야."}]
    )
    print("OpenAI 연결 성공!")
    print("답변:", response.choices[0].message.content)
except Exception as e:
    print("연결 실패:", e)

OpenAI 연결 성공!
답변: 안녕하세요! 연결 테스트 잘 되고 있는 것 같아요. 무엇을 도와드릴까요?


# 1. 시스템 페르소나 및 프롬프트 정의

In [4]:
# [프롬프트] 시스템 페르소나 정의 (SCM 전문가)
SCM_SYSTEM_PROMPT = """
당신은 SCM(공급망 관리) 전문 AI 어시스턴트입니다.
사용자의 질문을 분석하여 다음 두 가지 핵심 정보를 추출하는 것이 목표입니다.

1. **대상 품목 (Target Item)**: 사용자가 언급한 물품명 (예: 노트북, 모니터 등). 언급이 없으면 null.
2. **재고 관리 리스크 성향 (Risk Attitude)**: 사용자의 의도를 파악하여 '서비스 계수(Z값)'를 매핑.

### [Z값 매핑 가이드라인]
1. **보수적 운영 (Conservative)**:
   - 키워드: "절대 결품 안 됨", "수업 지장 없이", "매우 중요", "넉넉하게", "여유 있게", "평소대로".
   - **Z값: 1.65** (서비스 수준 약 95% 확보 - 안전 제일)

2. **공격적 운영 (Aggressive)**:
   - 키워드: "재고 최소화", "예산 삭감", "공간 부족", "타이트하게", "비용 절감", "0으로".
   - **Z값: 0.0** (안전버퍼 없음 - 비용 절감 최우선)

### [출력 형식 (JSON Only)]
반드시 아래 키를 포함한 JSON 형식으로만 응답하세요:
{
  "target_item": "추출된 물품명 또는 null",
  "intent_category": "Conservative" | "Aggressive",
  "z_value": 1.65 | 0.0,
  "reasoning": "선택 근거 요약 (한국어)"
}
"""

In [5]:
import json
import sys
import os
from openai import OpenAI
from google.colab import userdata

# [설정] Colab Secrets에서 API 키 가져오기
try:
    api_key = userdata.get('OPENAI_API_KEY')
    if not api_key:
        raise ValueError("API Key가 비어있습니다! (왼쪽 열쇠 아이콘 확인)")

    os.environ['OPENAI_API_KEY'] = api_key
    print("API Key 로드 성공")

    # 키가 있어야 클라이언트를 만들 수 있음
    client = OpenAI(api_key=api_key)

except Exception as e:
    print(f"[치명적 오류] API Key 설정 실패: {e}")
    # 여기서 프로그램을 아예 꺼버립니다. (더 이상 진행 안 함)
    sys.exit(1)

# [함수] LLM 호출 및 파라미터 추출
def extract_query_parameters(user_query, model="gpt-4o"):
    """
    어떤 일이 있어도 무조건 4가지 키(target_item, z_value, intent_category, reasoning)를 반환함
    """

    # [약속] 기본 반환값 틀 (실패하거나 값이 없으면 이게 나감)
    default_result = {
        "target_item": None,
        "z_value": 1.65,      # 기본값 (Conservative)
        "intent_category": "Conservative (Fallback)",
        "reasoning": "분석 실패 또는 기본값 적용"
    }

    try:
        # LLM 호출
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": SCM_SYSTEM_PROMPT},
                {"role": "user", "content": user_query}
            ],
            response_format={"type": "json_object"},
            temperature=0.0
        )

        # 응답 파싱
        content = response.choices[0].message.content
        result = json.loads(content)

        # LLM이 준 값으로 기본 틀을 덮어씌움 (update)
        final_result = default_result.copy()
        final_result.update(result)

        # z_value가 없을 경우 안전장치 한 번 더
        if 'z_value' not in final_result or final_result['z_value'] is None:
             final_result['z_value'] = 1.65

        return final_result

    except Exception as e:
        print(f"쿼리 분석 중 에러 발생: {e}")
        return default_result

API Key 로드 성공


In [6]:
# [테스트] 실행 결과 확인
if __name__ == "__main__":
    scenarios = [
        "이번 학기 노트북은 수업 중에 고장 나면 큰일 나니까 무조건 넉넉하게 잡아줘.",
        "올해 예산이 너무 삭감됐어. 모니터 재고 비용을 0으로 만들고 싶어.",
        "다기능복사기는 그냥 평소대로 운영해."
    ]

    print("-" * 110)
    print(f"{'Item':<10} | {'Intent':<12} | {'Z-Value':<8} | {'Reasoning'}")
    print("-" * 110)

    for query in scenarios:
        result = extract_query_parameters(query)

        item_disp = result['target_item'] if result['target_item'] else "전체"
        print(f"{item_disp:<10} | {result['intent_category']:<12} | {str(result['z_value']):<8} | {result['reasoning']}")

--------------------------------------------------------------------------------------------------------------
Item       | Intent       | Z-Value  | Reasoning
--------------------------------------------------------------------------------------------------------------
노트북        | Conservative | 1.65     | 사용자가 '수업 중에 고장 나면 큰일', '무조건 넉넉하게'라는 표현을 사용하여 노트북의 재고를 보수적으로 운영하고자 함을 나타냈습니다.
모니터        | Aggressive   | 0.0      | 사용자가 '예산 삭감'과 '재고 비용을 0으로'라는 표현을 사용하여 공격적인 재고 관리 방식을 원하고 있음을 나타냅니다.
다기능복사기     | Conservative | 1.65     | 사용자가 '평소대로'라는 표현을 사용하여 보수적 운영을 선호함을 나타냈습니다.


# 2. 안전버퍼 산출 함수 구현

In [7]:
import math
from datetime import datetime, timedelta

def get_lead_time_days(grade):
    """
    [DB 연동] 리드타임 등급을 실제 일수로 변환
    Grade 0: 1주 (7일) / Grade 1: 3주 (20일) / Grade 2: 6주 (38일)
    """
    mapping = {0: 7, 1: 20, 2: 38}
    return mapping.get(grade, 7) # 기본값 7일

def calculate_safety_metrics(z_value, model_rmse, lead_time_grade, predicted_failure_date_str):
    """
    Z값과 모델 오차(RMSE)를 이용해 '안전버퍼(일수)'를 계산하고,
    최종 '권장 발주일(D-Day)'을 산출하는 함수
    """
    # 1. 날짜 포맷 변환 (String -> Datetime)
    try:
        pred_date = datetime.strptime(predicted_failure_date_str, "%Y-%m-%d")
    except ValueError:
        return {"error": "Invalid date format"}

    # 2. 리드타임(조달 기간) 가져오기
    lt_days = get_lead_time_days(lead_time_grade)

    # 3. 안전버퍼(Safety Buffer) 계산 [핵심 로직]
    # 공식: Buffer(일) = Z * RMSE(일)
    # 소수점은 올림(ceil) 처리하여 안전하게 하루라도 더 확보
    buffer_days = math.ceil(z_value * model_rmse)

    # 4. 총 준비 기간 (리드타임 + 안전버퍼)
    total_prep_days = lt_days + buffer_days

    # 5. 권장 발주일(Reorder Point) 계산
    # 예측된 고장일로부터 준비 기간만큼 거꾸로 계산
    recommend_date = pred_date - timedelta(days=total_prep_days)

    # 6. 결과 리포트 생성
    return {
        "z_used": z_value,
        "rmse_used": model_rmse,
        "buffer_days": buffer_days,      # AI가 추가한 안전 기간
        "lead_time_days": lt_days,       # 물건 오는데 걸리는 시간
        "total_prep_days": total_prep_days,
        "predicted_failure": predicted_failure_date_str,
        "recommend_order_date": recommend_date.strftime("%Y-%m-%d")
    }

In [8]:
# [테스트] 시나리오별 결과 확인
if __name__ == "__main__":
    # 가정: 하이브리드 모델이 예측한 오차(RMSE)가 '5일' 정도임
    current_model_rmse = 5.0

    # 가정: AI가 예측한 이 장비의 고장 날짜
    ai_predicted_date = "2026-03-20"

    # 가정: 이 장비(노트북)는 해외 배송이라 리드타임 등급이 '1(20일)'임
    item_lead_time_grade = 1

    # 1단계에서 확정한 Z값 적용 (Conservative: 1.65 / Standard: 0.84 / Aggressive: 0.0)
    scenarios = [
        {"intent": "보수적 (1.65)", "z": 1.65},
        {"intent": "공격적 (0.0)",  "z": 0.0}
    ]

    print(f"■ 모델 오차(RMSE): {current_model_rmse}일 / 예측 고장일: {ai_predicted_date} / 리드타임: 20일")
    print("-" * 110)
    print(f"{'성향(Intent)':<15} | {'Z값':<5} | {'안전버퍼(일)':<12} | {'총 준비기간':<12} | {'★ 권장 발주일 (Order Date)'}")
    print("-" * 110)

    for case in scenarios:
        result = calculate_safety_metrics(
            z_value=case['z'],
            model_rmse=current_model_rmse,
            lead_time_grade=item_lead_time_grade,
            predicted_failure_date_str=ai_predicted_date
        )

        print(f"{case['intent']:<15} | {result['z_used']:<5} | {str(result['buffer_days']) + '일':<12} | {str(result['total_prep_days']) + '일':<12} | {result['recommend_order_date']}")

■ 모델 오차(RMSE): 5.0일 / 예측 고장일: 2026-03-20 / 리드타임: 20일
--------------------------------------------------------------------------------------------------------------
성향(Intent)      | Z값    | 안전버퍼(일)      | 총 준비기간       | ★ 권장 발주일 (Order Date)
--------------------------------------------------------------------------------------------------------------
보수적 (1.65)      | 1.65  | 9일           | 29일          | 2026-02-19
공격적 (0.0)       | 0.0   | 0일           | 20일          | 2026-02-28


# 3. 인사이트 생성 및 리포트 포맷팅

In [9]:
import json
import os
from openai import OpenAI
from google.colab import userdata

# API 키 로드 (기존 방식 유지)
try:
    api_key = userdata.get('OPENAI_API_KEY')
    client = OpenAI(api_key=api_key)
except Exception as e:
    print(f"API Key 설정 확인 필요: {e}")

# 3. 인사이트 생성 및 리포트 작성 프롬프트 (수정됨: 시간/날짜 중심)
REPORT_SYSTEM_PROMPT = """
당신은 대학 자산 관리 실무자(User)를 돕는 'SCM AI 분석 파트너'입니다.
AI가 계산한 **'발주 시점 데이터'**를 바탕으로, 실무자가 **'언제 주문해야 하는지'** 명확한 행동 지침을 제공하세요.

[입력 데이터]
1. 품목명: 관리 대상 (예: 노트북)
2. 전략 (Z값): 사용자가 선택한 리스크 성향 (보수적일수록 발주 시점을 당김)
3. 산출된 안전버퍼(일수): 예측 오차를 고려하여 며칠을 더 미리 주문하는지
4. 권장 발주일(D-Day): AI가 계산한 최종 주문 데드라인

[작성 가이드]
- **독자:** 실제 발주를 넣는 '자산 관리 담당자'
- **톤앤매너:** 명확하고 정중한 '해요체'. (예: "안전버퍼를 9일 확보하여 2월 19일까지 발주하시길 권장합니다.")
- **핵심 내용:**
  - 사용자의 성향(Z값) 때문에 안전기간을 얼마나 더 잡았는지 설명.
  - **'권장 발주일'**을 강조하여 결품(Go-Dark)을 막도록 유도.

[출력 형식 (JSON)]
반드시 아래 키를 포함한 **JSON 형식**으로만 응답하세요:
{
  "report_title": "직관적인 제목 (예: [D-7] 노트북 발주 시점 도래 알림)",
  "analysis_summary": "상황 분석 (예측 오차와 리스크 성향을 고려한 버퍼 설명)",
  "action_item": "구체적 실행 가이드 (날짜 포함)",
  "alert_level": "Normal" | "High" | "Critical"
}
"""

def generate_scm_report(item_name, strategy_info, step2_result):
    """
    [Step 3] 계산된 결과(Step 2)를 받아서 리포트 문구(Text)를 생성하는 함수
    """
    # LLM에게 던져줄 상황 요약 정보 (시간 관점으로 변경)
    context_input = f"""
    - 품목명: {item_name}
    - 사용자의 전략: {strategy_info['intent_category']} (Z값: {strategy_info['z_value']})
    - 모델 오차(RMSE): {step2_result['rmse_used']}일 (이만큼 날짜가 틀릴 수 있음)
    - 확보한 안전버퍼: {step2_result['buffer_days']}일 (오차를 커버하기 위해 앞당긴 기간)
    - 총 조달 기간(리드타임 포함): {step2_result['total_prep_days']}일
    - ★ AI 권장 발주 날짜: {step2_result['recommend_order_date']}
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini", # 가성비 모델 사용
            messages=[
                {"role": "system", "content": REPORT_SYSTEM_PROMPT},
                {"role": "user", "content": context_input}
            ],
            response_format={"type": "json_object"},
            temperature=0.7 # 문장 생성이므로 약간의 창의성 허용
        )
        return json.loads(response.choices[0].message.content)
    except Exception as e:
        print(f"리포트 생성 중 에러 발생: {e}")
        return None

In [10]:
# [테스트] Step 1, 2, 3 전체 연결 시뮬레이션
if __name__ == "__main__":
    print("🚀 [Step 3] AI 리포트 생성 테스트 시작...\n")

    # [가정] Step 1 결과 (사용자: 보수적)
    mock_step1_strategy = {"intent_category": "Conservative", "z_value": 1.65}

    # [가정] Step 2 결과 (계산된 날짜들)
    # 아까 2단계 코드에서 나온 결과를 그대로 흉내 냄
    mock_step2_result = {
        "rmse_used": 5.0,
        "buffer_days": 9,             # 1.65 * 5.0 = 8.25 -> 9일 올림
        "total_prep_days": 29,        # 리드타임 20 + 버퍼 9
        "recommend_order_date": "2026-02-19" # 오늘보다 미래라고 가정
    }

    # [실행] 리포트 생성
    final_report = generate_scm_report(
        item_name="노트북(교육용)",
        strategy_info=mock_step1_strategy,
        step2_result=mock_step2_result
    )

    # [결과 출력]
    if final_report:
        print(f"📄 제목: {final_report['report_title']}")
        print(f"📊 분석: {final_report['analysis_summary']}")
        print(f"✅ 조치: {final_report['action_item']}")
        print(f"🚨 레벨: {final_report['alert_level']}")
    else:
        print("리포트 생성 실패")

🚀 [Step 3] AI 리포트 생성 테스트 시작...

📄 제목: [D-9] 노트북(교육용) 발주 시점 도래 알림
📊 분석: 사용자의 보수적 리스크 성향(Z값: 1.65)으로 인해, 예측 오차(RMSE) 5일을 고려하여 9일의 안전버퍼를 확보하였습니다. 이를 통해 결품 상황을 방지하고자 발주 시점을 앞당겼습니다.
✅ 조치: 2026년 2월 19일까지 노트북(교육용)을 발주하시길 권장합니다.
🚨 레벨: Normal


# 4. 시나리오 테스트 및 정합성 검증

In [11]:
import math
import time

# 통합 시뮬레이션 및 검증 함수
def run_full_simulation(user_query, item_name, model_rmse, lead_time_grade, predicted_date):
    print("\n" + "=" * 80)
    print(f"🎬 [시나리오 테스트] 사용자 질문: \"{user_query}\"")
    print("=" * 80)

    # 1단계: 사용자 의도 파악 (Intent Recognition)
    print("\n▶ 1단계: 의도 및 Z값 추출 중...")
    try:
        # 1단계 함수 호출
        step1_result = extract_query_parameters(user_query)

        # 결과 출력
        z_val = step1_result['z_value']
        intent = step1_result['intent_category']
        print(f"   ㄴ 결과: 성향={intent}, Z값={z_val}")

    except Exception as e:
        print(f"!! 1단계 에러: {e}")
        return

    # 2단계: 안전버퍼 및 발주일 계산 (Calculation)
    print(f"\n▶ 2단계: 안전버퍼(일수) 계산 중... (RMSE: {model_rmse}일)")
    try:
        # 2단계 함수 호출
        step2_result = calculate_safety_metrics(
            z_value=z_val,
            model_rmse=model_rmse,
            lead_time_grade=lead_time_grade,
            predicted_failure_date_str=predicted_date
        )

        # 결과 출력
        buffer_days = step2_result['buffer_days']
        rec_date = step2_result['recommend_order_date']
        print(f"   ㄴ 결과: 안전버퍼={buffer_days}일, 권장발주일={rec_date}")

    except Exception as e:
        print(f"!! 2단계 에러: {e}")
        return

    # [검증 포인트] Python 계산이 정확한지 기계적으로 체크 (Assert)
    expected_buffer = math.ceil(z_val * model_rmse)
    if buffer_days == expected_buffer:
        print(" [Pass] 데이터 정합성 검증 성공 (Python 계산 일치)")
    else:
        print(f"[Fail] 데이터 불일치! (계산값: {expected_buffer} vs 함수반환값: {buffer_days})")

    # 3단계: 리포트 생성 (Report Generation)
    print("\n▶ 3단계: 최종 리포트 생성 중...")
    try:
        # 3단계 함수 호출 (2단계 결과를 입력으로 사용)
        final_report = generate_scm_report(item_name, step1_result, step2_result)

        if final_report:
            print("\n" + "■" * 25 + " [최종 AI 리포트] " + "■" * 25)
            print(f"📄 제목:  {final_report.get('report_title')}")
            print(f"📊 분석:  {final_report.get('analysis_summary')}")
            print(f"✅ 조치:  {final_report.get('action_item')}")
            print(f"🚨 레벨:  {final_report.get('alert_level')}")
            print("-" * 70)
        else:
            print("!! 3단계 리포트 생성 실패")

    except Exception as e:
        print(f"!! 3단계 에러: {e}")

In [12]:
# [실행] 가상 시나리오 테스트
if __name__ == "__main__":
    # 변수 설정
    target_item = "고성능 워크스테이션"
    mock_rmse = 4.5
    mock_lt_grade = 1
    mock_pred_date = "2026-03-15"

    # 질문 설정
    input_query = "이번 학기 컴퓨터공학과 실습실 장비 교체해야 해? 수업 지장 없게 넉넉하게 잡아줘."

    # 실행
    run_full_simulation(input_query, target_item, mock_rmse, mock_lt_grade, mock_pred_date)


🎬 [시나리오 테스트] 사용자 질문: "이번 학기 컴퓨터공학과 실습실 장비 교체해야 해? 수업 지장 없게 넉넉하게 잡아줘."

▶ 1단계: 의도 및 Z값 추출 중...
   ㄴ 결과: 성향=Conservative, Z값=1.65

▶ 2단계: 안전버퍼(일수) 계산 중... (RMSE: 4.5일)
   ㄴ 결과: 안전버퍼=8일, 권장발주일=2026-02-15
 [Pass] 데이터 정합성 검증 성공 (Python 계산 일치)

▶ 3단계: 최종 리포트 생성 중...

■■■■■■■■■■■■■■■■■■■■■■■■■ [최종 AI 리포트] ■■■■■■■■■■■■■■■■■■■■■■■■■
📄 제목:  [D-8] 고성능 워크스테이션 발주 시점 도래 알림
📊 분석:  사용자의 보수적인 리스크 성향(Z값: 1.65)으로 인해, 예측 오차 4.5일을 감안하여 8일의 안전버퍼를 추가로 확보하였습니다. 이는 발주 시점을 보다 앞당겨 결품을 방지하기 위한 조치입니다.
✅ 조치:  안전버퍼를 8일 확보하여 2월 7일까지 고성능 워크스테이션을 발주하시길 권장합니다.
🚨 레벨:  Normal
----------------------------------------------------------------------
